https://gist.github.com/qpwo/c538c6f73727e254fdc7fab81024f6e1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import count
from collections import namedtuple
import os
import json
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Categorical
%matplotlib inline

In [2]:
from mcts import MCTS, KlondikeNode

In [3]:
from tuplestate import init_from_solvitaire
from benchmarking import convert_shootme_to_solvitaire_json


def listdir(path):
    ls = os.listdir(path)
    return [os.path.join(path, f) for f in ls]

solved = listdir('./fixtures/shootme/solved/')
solvedmin = listdir('./fixtures/shootme/solvedmin/')
fixtures = solved + solvedmin

def filename_to_klonstate(fname):
    with open(fname) as f:
        solv = convert_shootme_to_solvitaire_json(f.read())
        state = init_from_solvitaire(solv)
    return state

def random_state():
    fname = random.choice(fixtures)
    state = filename_to_klonstate(fname)
    return KlondikeNode(*state)

In [4]:
random.seed(0)
root_state = random_state()
tree = MCTS()
state = root_state

In [5]:
for i in range(2):
    print(i, end=" ")
    for _ in range(50):
        tree.do_rollout(state)
    state = tree.choose(state)
    if state.is_terminal():
        print("TERMINAL")
        print(state.to_pretty_string())

0 1 

In [7]:
import cProfile
cProfile.run('for _ in range(20): tree.do_rollout(state)', 'restats')

In [8]:
import pstats
from pstats import SortKey
p = pstats.Stats('restats')
p.sort_stats(SortKey.CUMULATIVE).print_stats()

Tue Dec  3 01:10:28 2019    restats

         766848 function calls in 2.077 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.077    2.077 {built-in method builtins.exec}
        1    0.000    0.000    2.077    2.077 <string>:1(<module>)
       20    0.000    0.000    2.077    0.104 /Users/glifchits/Development/KlonSolve/mcts.py:43(do_rollout)
       20    0.003    0.000    2.061    0.103 /Users/glifchits/Development/KlonSolve/mcts.py:80(_simulate)
     5224    0.004    0.000    2.040    0.000 /Users/glifchits/Development/KlonSolve/mcts.py:169(is_terminal)
     7864    0.049    0.000    2.031    0.000 /Users/glifchits/Development/KlonSolve/mcts.py:137(find_children)
     5244    0.007    0.000    2.029    0.000 /Users/glifchits/Development/KlonSolve/mcts.py:163(is_childless)
    22386    0.042    0.000    1.695    0.000 /Users/glifchits/Development/KlonSolve/mcts.py:133(make_move)
    22386